<a href="https://colab.research.google.com/github/aburchell/pollack-notebooks/blob/main/pollack_data_tidying.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import glob
import math
import random
import re
from plotnine import ggplot, aes, geom_density, geom_histogram, facet_wrap, geom_bar, geom_point, xlab, ylab, ggtitle, qplot, coord_fixed, geom_jitter, facet_grid, facet_wrap, geom_abline, geom_line, geom_errorbar
from plotnine.options import set_option
from plotnine.scales import scale_color_brewer, scale_color_manual
from plotnine.themes import theme, element_rect, element_line, themeable
from plotnine.labels import labs
from sklearn.metrics import mutual_info_score, adjusted_mutual_info_score, normalized_mutual_info_score
from sklearn.feature_selection import mutual_info_classif
import pandas as pd
import numpy as np
from numpy import std
import altair as alt
import sklearn as skl
import scipy as sp

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load local data

In [ ]:
# Load all CSVs from directory
from os import walk

def load_cvs(directory):
    _, _, filenames = next(walk(directory))
    datafiles = [(directory + '/' + file) #.replace(" ", "_")) 
                 for file in filenames if file[-3:] == 'csv']
    return datafiles

In [ ]:
# datafiles = load_cvs("./drive/MyDrive/data")
datafiles = [filename for filename in load_cvs("./drive/MyDrive/data") if ('_500' not in filename) and ('short' not in filename)]
datafiles

['./drive/MyDrive/data/AF tone id data.csv',
 './drive/MyDrive/data/ET tone id data.csv',
 './drive/MyDrive/data/RW tone id data.csv',
 './drive/MyDrive/data/AB tone id data.csv',
 './drive/MyDrive/data/RG tone id data.csv',
 './drive/MyDrive/data/DC tone id data.csv',
 './drive/MyDrive/data/JN tone id data.csv',
 './drive/MyDrive/data/MB tone data.csv',
 './drive/MyDrive/data/RK tone id data.csv',
 './drive/MyDrive/data/AB_16_2500.csv',
 './drive/MyDrive/data/RK_16_2500.csv',
 './drive/MyDrive/data/AF_16_2500.csv',
 './drive/MyDrive/data/AF_just16_2500.csv',
 './drive/MyDrive/data/RK_just16_2500.csv',
 './drive/MyDrive/data/DC_just16_2500.csv',
 './drive/MyDrive/data/AB_just16_2500.csv',
 './drive/MyDrive/data/AF tone id data a.csv',
 './drive/MyDrive/data/AB_16_2500_a.csv']

In [ ]:
# pid = participant ID
def get_df_with_pid(fname, source_file_id):
    participant_label = re.search('data/([A-Z]{2}[0-9]*).*\.csv$',fname).group(1)
    df = pd.read_csv(fname)
    df['pid'] = participant_label
    df['source_file_id'] = source_file_id
    return df
    
local_df = pd.concat([get_df_with_pid(filename, i) for i, filename in enumerate(datafiles)])
local_df

,sender,sender_type,sender_id,timestamp,meta,Unnamed: 5,confidence,consent,correct,correctResponse,debrief,desired_repetitions_per_tone,duration,ended_on,experience,headphone,ntones,pitch,response,response_action,time_commit,time_end,time_render,time_run,time_show,time_switch,tone_id,tone_label,tones,tones_played,url,pid,source_file_id,block_id
0,consent,html.Form,0,2021-01-25T21:37:37.333Z,"{""labjs_version"":""20.1.1"",""labjs_build"":{""flav...",NaN,NaN,True,NaN,NaN,NaN,NaN,138708.295,form submission,NaN,NaN,NaN,NaN,NaN,NaN,139586.385,139578.125,836.357,1006.620,869.830,NaN,NaN,NaN,NaN,NaN,{},AF,0,NaN
1,instructions,html.Page,1_0,2021-01-25T21:37:46.679Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7933.635,response,NaN,NaN,NaN,NaN,NaN,click,148932.560,148903.045,139418.254,141126.175,140969.410,148825.935,NaN,NaN,NaN,NaN,NaN,AF,0,NaN
2,slider for volume,html.Screen,1_1,2021-01-25T21:37:52.808Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6234.810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155061.195,155060.745,148775.967,148933.090,148825.935,154931.020,NaN,NaN,NaN,NaN,NaN,AF,0,NaN
3,ISI,canvas.Screen,1_2,2021-01-25T21:37:53.674Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,834.628,timeout,NaN,NaN,NaN,NaN,NaN,NaN,155927.900,155926.940,154881.083,155061.505,154931.020,155765.648,NaN,NaN,NaN,NaN,NaN,AF,0,NaN
4,Demo start message,canvas.Screen,1_3_0_0_0,2021-01-25T21:37:57.398Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3883.017,response,NaN,NaN,6.0,NaN,NaN,click,159651.330,159648.665,155926.940,155928.465,155765.648,159534.899,NaN,"{""1"":1,""2"":2,""3"":3,""4"":4,""5"":5,""6"":6,""7"":7,""8""...","{""2"":[100,8000],""3"":[100,894,8000],""4"":[100,43...","{""2"":{""1"":0,""2"":0},""3"":{""1"":0,""2"":0,""3"":0},""4""...",NaN,AF,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2283,demographics questions,flow.Sequence,2_4,2021-03-10T19:43:16.841Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17459.300,completion,NaN,NaN,NaN,NaN,NaN,NaN,1885716.000,1885715.000,1868206.680,1868218.000,1868255.700,1885734.700,NaN,NaN,NaN,NaN,NaN,AB,17,NaN
2284,debrief,html.Form,2_5,2021-03-10T19:43:21.019Z,NaN,NaN,NaN,NaN,NaN,NaN,yes,NaN,4160.300,form submission,NaN,NaN,NaN,NaN,NaN,NaN,1889895.000,1889895.000,1885717.980,1885716.000,1885734.700,1889921.780,NaN,NaN,NaN,NaN,NaN,AB,17,NaN
2285,If consent,flow.Sequence,2,2021-03-10T19:43:21.019Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,completion,NaN,NaN,NaN,NaN,NaN,NaN,1889895.000,1889895.000,NaN,10130.000,NaN,1889921.780,NaN,NaN,NaN,NaN,NaN,AB,17,NaN
2286,end,canvas.Screen,3,2021-03-10T19:43:24.325Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3266.220,response,NaN,NaN,NaN,NaN,NaN,click,1893201.000,1893188.000,1889904.200,1889896.000,1889921.780,1893206.740,NaN,NaN,NaN,NaN,NaN,AB,17,NaN


# Loading JATOS data

In [ ]:
jatos_dfs = []
for i, jatos_file in enumerate(glob.glob("/content/drive/MyDrive/data/jatos_results_202103*.json")):
  this_df = pd.read_json(jatos_file, lines=True)
  this_df['source_file_id'] = [f'j_{i}']*len(this_df.index)
  jatos_dfs.append(this_df)
jatos_df = pd.concat(jatos_dfs).reset_index()
jatos_df
# [filename for _, _,filename in next(walk("./drive/MyDrive/data"))[0]]
#jatos_df = pd.read_json('/content/drive/MyDrive/data/jatos_results_20210203105606.txt')

,index,url,meta,sender,sender_type,sender_id,ended_on,duration,time_run,time_render,time_show,time_end,time_commit,timestamp,time_switch,consent,,response,response_action,ntones,block_id,desired_repetitions_per_tone,tone_id,correctResponse,correct,confidence,experience,headphone,pitch,debrief,source_file_id
0,0,{'srid': '656'},"{'labjs_version': '20.2.2', 'labjs_build': {'f...",Safari check,html.Page,0.0,timeout,199.902,1504.270,1495.299,1511.957,1695.204,1696.660,2021-03-10 00:49:42.775000+00:00,1711.859,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,j_0
1,1,NaN,NaN,consent,html.Form,1.0,form submission,13416.801,1696.935,1695.204,1711.859,15128.660,15128.950,2021-03-10 00:49:56.207000+00:00,NaN,true,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,j_0
2,2,NaN,NaN,instructions,html.Page,20.0,response,10215.205,16865.865,15111.743,16861.615,27076.820,27078.825,2021-03-10 00:50:08.157000+00:00,27094.588,NaN,NaN,,click,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,j_0
3,3,NaN,NaN,slider for volume,html.Screen,21.0,None,8028.407,27078.920,27077.924,27094.588,35122.995,35123.195,2021-03-10 00:50:16.201000+00:00,35127.712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,j_0
4,4,NaN,NaN,ISI,canvas.Screen,22.0,timeout,999.648,35124.435,35094.378,35127.712,36110.700,36111.825,2021-03-10 00:50:17.190000+00:00,36127.360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,j_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13723,2283,NaN,NaN,demographics questions,flow.Sequence,24.0,completion,56841.098,1660094.315,1660091.342,1660107.932,1716949.030,1716949.460,2021-03-25 06:14:32.378000+00:00,1716963.047,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,j_5
13724,2284,NaN,NaN,debrief,html.Form,25.0,form submission,10729.673,1716949.570,1716946.366,1716963.047,1727692.720,1727692.965,2021-03-25 06:14:43.121000+00:00,1727700.361,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,j_5
13725,2285,NaN,NaN,If consent,flow.Sequence,2.0,completion,NaN,15081.585,NaN,NaN,1727692.720,1727693.190,2021-03-25 06:14:43.122000+00:00,1727700.361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,j_5
13726,2286,NaN,NaN,end,canvas.Screen,3.0,response,30071.339,1727693.295,1727683.687,1727700.361,1757771.700,1757772.590,2021-03-25 06:15:13.201000+00:00,1757778.513,NaN,NaN,,click,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,j_5


### Adding pid to JATOS data
We would like a usuable participant term, ie an id identifying the participant on each row

In [ ]:
pids = []
srid = None
for i in range(len(jatos_df['url'])):
  if (srid != jatos_df.loc[i, 'url']) and type(jatos_df['url'][i]) == dict:
    srid = jatos_df['url'][i]
  pids.append(srid['srid'])
jatos_df['pid'] = pids

### sender_id discrepency??
For some reason (maybe in the read_json step), the sender_id variable from the JATOS data are formatted as all smushed together, rather than an underscore between parts of the id. This should be okay for us though, as we have only 8 blocks, ie only one digit [0-7]. I believe the third digit should be the equal to block_id.

In [ ]:
jatos_df[jatos_df.sender == 'Get response']['sender_id']

8         230111.0
14        230121.0
21        230141.0
27        230151.0
34        230171.0
           ...    
13676    2381651.0
13682    2381661.0
13691    2381701.0
13700    2381741.0
13707    2381761.0
Name: sender_id, Length: 1830, dtype: float64

# Combining local and JATOS data

In [ ]:
df = pd.concat([local_df, jatos_df])

# Populating demographic questions

In [ ]:
experience, confidence, headphone, perfect_pitch = [], [], [], []
for (pid, source_file_id), data in df.groupby(['pid', 'source_file_id']):
  experience_lvl = int(pd.unique(data[data['experience'].notna()]['experience'])[0])
  confidence_lvl = int(pd.unique(data[data['confidence'].notna()]['confidence'])[0])
  headphones_use = pd.unique(data[data['headphone'].notna()]['headphone'])[0]
  pitch = pd.unique(data[data['pitch'].notna()]['pitch'])[0]

  experience.extend([experience_lvl]*len(data.index)) 
  confidence.extend([confidence_lvl]*len(data.index)) 
  headphone.extend([headphones_use]*len(data.index)) 
  perfect_pitch.extend([pitch]* len(data.index))
df['experience'] = experience
df['confidence'] = confidence
df['headphone'] = headphone
df['pitch'] = perfect_pitch

# Getting just the users' responses

In [ ]:
response_df = df[df.sender == 'Get response'].reset_index()

In [ ]:
data_types_dict = {'response': int, 'correctResponse': int, 'ntones': int}
response_df = response_df.astype(data_types_dict)
response_df.ntones = pd.Categorical(response_df.ntones)

response_df.name = 'response'

### correct = response == correctResponse
That sort of says it all, really. Correct is a boolean, reponse is the button the participant actually clicked, correctResponse is the button they should have pressed.

# Performance across block number
The 'block_id' variable is NaN for a lot of rows, ie the early ones.
Maybe this is because that data was from before we added a block index to the experiment.

We can get around this by extracting out the block id from the `sender_id` variable.


In [ ]:
response_df['block_sender_id'] = [int(str(int(sender_id))[2])+1 for sender_id in response_df.sender_id]

In [ ]:
response_df.drop(columns=['level_0', 'meta', 'tone_label', 'tones', 'tones_played'], inplace=True)
response_df.head()

,sender,sender_type,sender_id,timestamp,Unnamed: 5,confidence,consent,correct,correctResponse,debrief,desired_repetitions_per_tone,duration,ended_on,experience,headphone,ntones,pitch,response,response_action,time_commit,time_end,time_render,time_run,time_show,time_switch,tone_id,url,pid,source_file_id,block_id,index,,block_sender_id
0,Get response,html.Page,1_3_0_2_0_1,2021-01-25T21:38:05.127Z,NaN,6,NaN,False,3,NaN,7.0,2582.226,response,0,yes,6,no,5,click button#button5,167381.085,167355.340,164756.030,164907.230,164773.114,167241.279,3.0,NaN,AF,0,NaN,NaN,NaN,1
1,Get response,html.Page,1_3_0_2_1_1,2021-01-25T21:38:11.430Z,NaN,6,NaN,False,6,NaN,7.0,918.365,response,0,yes,6,no,5,click button#button5,173683.175,173680.945,172745.827,172898.290,172762.580,173563.152,6.0,NaN,AF,0,NaN,NaN,NaN,1
2,Get response,html.Page,1_3_0_2_3_1,2021-01-25T21:38:16.448Z,NaN,6,NaN,True,1,NaN,7.0,1032.596,response,0,yes,6,no,1,click button#button1,178702.065,178699.915,177824.750,177826.500,177667.319,178567.329,1.0,NaN,AF,0,NaN,NaN,NaN,1
3,Get response,html.Page,1_3_0_2_5_1,2021-01-25T21:38:21.777Z,NaN,6,NaN,False,5,NaN,7.0,1407.960,response,0,yes,6,no,3,click button#button3,184031.125,184028.770,182604.010,182755.785,182620.810,183888.477,5.0,NaN,AF,0,NaN,NaN,NaN,1
4,Get response,html.Page,1_3_0_2_6_1,2021-01-25T21:38:27.527Z,NaN,6,NaN,True,4,NaN,7.0,1453.635,response,0,yes,6,no,4,click button#button4,189781.830,189779.845,188483.420,188485.035,188326.210,189660.637,4.0,NaN,AF,0,NaN,NaN,NaN,1


# Export data

In [ ]:
#for chunk in np.array_split(response_df, 10000):
  #print(chunk.to_csv())
response_df.to_csv('response_data_4_19_21.csv')
!cp response_data_4_19_21.csv "./drive/MyDrive/data/exports"